In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam

# Load text dataset (e.g., Shakespeare Sonnets)
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Preprocessing
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Create training examples and targets
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Model definition
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=None),
    LSTM(rnn_units, return_sequences=True, stateful=False, recurrent_initializer='glorot_uniform'),
    Dense(vocab_size)
])

# Compile model
model.compile(optimizer=Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Training the model
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

# Text generation
def generate_text(model, start_string, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for i in range(1000):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generate new text
print(generate_text(model, start_string="ROMEO: ", temperature=0.5))


Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 16s 73ms/step - loss: 2.9280
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - loss: 1.8955
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.6239
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.4832
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - loss: 1.3933
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 72ms/step - loss: 1.3330
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 74ms/step - loss: 1.2880
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - loss: 1.2428
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.2057
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.1666
Epoch 11/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.1295
Epoch 12/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - loss: 1.0856
Epoch 13/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.0441
Epoch 14/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 71ms/step - loss: 1.0039
Epoch 15/20
172/172 ━━━━━━━━━